# Exploratory Data Analysis

Let's create an InfluxDB client by importing the modules and setting all the required configuration. 

In [1]:
import os
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

# configure influxdb client
BUCKET = "racing"
ORG = "b4mad"
TOKEN = os.environ.get(
    "INFLUXDB_TOKEN",
    "citqAMr66LLb25hvaaZm2LezOc88k2ocOFJcJDR6QB-RmLJa_-sAr9kYB4vSFYaz8bt26lm7SokVgpQKdgKFKA==",
)
URL = "https://telemetry.b4mad.racing/"

# and create the client and a quary api
client = influxdb_client.InfluxDBClient(url=URL, token=TOKEN, org=ORG)

query_api = client.query_api()

In [4]:
SESSION = "1668412932"

query = f"""
from(bucket: "{BUCKET}")
  |> range(start: -10y, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "laps_cc")
  |> filter(fn: (r) => r["_field"] == "CurrentLap" or r["_field"] == "CurrentLapTime")
  |> filter(fn: (r) => r["SessionId"] == "{SESSION}")
  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> sort(columns: ["_time"], desc: false)
"""

records = query_api.query_stream(org=ORG, query=query)

current_lap = 1
maxLapTime = 0
for record in records:
    if record["CurrentLap"] != current_lap:
        print(f"lap {current_lap} finished with a time of {maxLapTime}")
        current_lap = record["CurrentLap"]
    maxLapTime = record["CurrentLapTime"]

lap 1 finished with a time of 135.6537
lap 2.0 finished with a time of 126.602333
lap 3.0 finished with a time of 126.9306
lap 4.0 finished with a time of 126.477264
lap 5.0 finished with a time of 126.411964
lap 6.0 finished with a time of 128.554626
lap 7.0 finished with a time of 127.8808
